In [147]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from globalSetting import globalParameters
import os

# import global parameter setting
gParameter = globalParameters()
trainingYrs = ['2010', '2011', '2012', '2013', '2014', '2015']
threshold = gParameter.threshold
conn = gParameter.conn
p_Tmin = gParameter.p_Tmin
p_Ninf = gParameter.p_Ninf

# Count and collect the training years of each postcode
def trainingCounts(cleaned_df, trainingPC6counts, simulationYear):
    pc6IDs = cleaned_df.iloc[:,0]
    for pc6 in pc6IDs:
        if bool(trainingPC6counts.get(pc6)):
            trained_yrs = trainingPC6counts[pc6]
            trained_yrs.append(simulationYear)
            trainingPC6counts[pc6] = trained_yrs
        else:
            trainingPC6counts[pc6] = [simulationYear]
    return trainingPC6counts


def dinstinctArchetype(cleaned_df):
    archetypes = cleaned_df.archetype.unique()
    return archetypes


# read pc6 averaged EUI from the precalculated csv file and return avgEUI in a dictionary
def readPC6_avgEUI():
    path = os.getcwd() + '/likelihood_model/pc6_measurements.csv'
    pc6_measurement_df = pd.read_csv(path, delimiter=';', index_col=False)
    pc6_avgEUI = {}
    for idx, pc6_data in pc6_measurement_df.iterrows():
        pc6_avgEUI[str(pc6_data.postcode)] = pc6_data["mean"]
    return pc6_avgEUI


def computelikelihood(meteredConsumption, nSimConsumption, Std):
    likelihood = norm.pdf(meteredConsumption, nSimConsumption, Std)
    return likelihood


def computePosterior(cleaned_df, pc6_name, pc6_metered, archetype, prior, gridSize):
    # return pc6 specific averaged EUI
    try:
        pc6_avgEUI = readPC6_avgEUI()[pc6_name]
    except: 
        pc6_avgEUI = pc6_metered
    # based on the residual analysis of the normalized categorial linear regression analysis (archetype, year)
    # residual distribution is close to a normal distribution with a standard devition equals to 6.88% of PC6 EUI.
    stdPercent = 0.0688
    pc6Std = pc6_avgEUI * stdPercent
    
    likelihoodXprior = []
    pc6_id = cleaned_df.index[cleaned_df['postcode'] == pc6_name]
    print "pc6: ", pc6_name, " pc6_id", pc6_id, " pc6std: ", pc6Std, " pc6_metered: ", pc6_metered

    for n in range(gridSize * gridSize):
        nSimConsumption = cleaned_df.loc[pc6_id, 'case' + str(n + 1) + '_kwh_m3']
        nLikelihood = computelikelihood(pc6_metered, nSimConsumption, pc6Std)
        # likelihoodXprior.append(nLikelihood*prior[n])
        # to avoid numerical problem, apply the principle: log(product(Pi))=sum(log(Pi))
        likelihoodXprior.append(np.log(nLikelihood) + np.log(prior[n]))

    likelihoodXprior = np.exp(likelihoodXprior)
    probD = sum(likelihoodXprior)
    pc6_posterior = np.multiply(likelihoodXprior, 100.0) / probD
    return pc6_posterior


def trainingPhase():
    gridSize = globalParameters().gridSize
    # initialize an empty dict() to record trained year count of each postcode
    trainingPC6counts = dict()
    # initialize an empty dict() to update prior to posterior during the training phase
    pc6_prob = dict()
    pc6_archetype = dict()

    for year in trainingYrs:
        # pre-processing input data
        path = gParameter.fileDirectory + gParameter.filePrefix + str(year) + '.csv'
        df = pd.read_csv(path, delimiter=',', index_col=False)
        # drop the unnammed column
        df = df.reset_index(drop=True).iloc[:,1:]

        # recording each PC6 has gone through how many years of training
        pc6TrainedCounts = trainingCounts(df, trainingPC6counts, year)

        for pc6, archetype, pc6_metered in zip(df.postcode[:], df.archetype[:], df.l_pc6_consumption_kwh_m3[:]):
            # if this is the first time the pc6 is trained, initialize equal prior probability,
            if pc6_prob.get(pc6) is None:
                pc6_prob[pc6] = [(1.0 / gridSize) ** 2] * (gridSize * gridSize)
                pc6_archetype[pc6] = archetype

            # compute posterior for each postcode of the year, and update the prior with posterior
            prior = pc6_prob[pc6]
            posterior = computePosterior(df, pc6, pc6_metered, archetype, prior, gridSize)
            pc6_prob[pc6] = posterior

    return pc6_prob, pc6TrainedCounts, pc6_archetype


def inputCombination(p_Tmin, p_Ninf):
    inputSets = []
    for p_temp in p_Tmin:
        for p_inf in p_Ninf:
            inputSets.append([p_temp, p_inf])
    return inputSets


def createPC6_posteriorInputsDB(conn):
    cur = conn.cursor()
    cur.execute(
        """
        DROP TABLE IF EXISTS public."pc6_posterior_results_likelihood_modified";

        CREATE TABLE public."pc6_posterior_results_likelihood_modified"
        (
        "postcode" character varying,
        "archetype" character varying,
        "maxProb" DOUBLE PRECISION ,
        "post_Tmin" DOUBLE PRECISION,
        "post_Ninf" DOUBLE PRECISION,
        "trainingTimes" integer
        );
        """
    )
    cur.close()
    conn.commit()


# pick up the most likely input combination from the joint posterior distribution
def pickUpInputCombination(pc6_prob, pc6TrainedCounts, pc6_archetype, inputSets, conn):
    # create or overwrite a table in DB to store calibrated information
    createPC6_posteriorInputsDB(conn)

    cur = conn.cursor()
    for pc6 in pc6_prob.keys():
        archetype = pc6_archetype[pc6]
        maxProb = max(pc6_prob[pc6].tolist())[0]
        maxProbID = pc6_prob[pc6].tolist().index(max(pc6_prob[pc6].tolist()))
        post_Tmin = inputSets[maxProbID][0]
        post_Ninf = inputSets[maxProbID][1]
        trainingTimes = len(pc6TrainedCounts[pc6])

        cur.execute(
        """INSERT INTO public."pc6_posterior_results_likelihood_modified"
           VALUES (%s, %s, %s, %s, %s, %s)""", [pc6, archetype, maxProb, post_Tmin, post_Ninf, trainingTimes]
        )
    cur.close()
    conn.commit()


def main():
    pc6_prob, pc6TrainedCounts, pc6_archetype = trainingPhase()

    inputSets = inputCombination(p_Tmin, p_Ninf)
    pickUpInputCombination(pc6_prob, pc6TrainedCounts, pc6_archetype, inputSets, conn)

In [148]:
pc6_prob, pc6TrainedCounts, pc6_archetype = trainingPhase()

pc6:  1097EZ  pc6_id Int64Index([0], dtype='int64')  pc6std:  20.7473722522  pc6_metered:  159.812928497
pc6:  1097HN  pc6_id Int64Index([1], dtype='int64')  pc6std:  17.7714794531  pc6_metered:  154.802541298
pc6:  1097HP  pc6_id Int64Index([2], dtype='int64')  pc6std:  15.979675824  pc6_metered:  138.230051331
pc6:  1097HR  pc6_id Int64Index([3], dtype='int64')  pc6std:  7.54305727552  pc6_metered:  61.6551481436
pc6:  1097HS  pc6_id Int64Index([4], dtype='int64')  pc6std:  5.82039922605  pc6_metered:  45.7823786298
pc6:  1097HT  pc6_id Int64Index([5], dtype='int64')  pc6std:  5.57286129805  pc6_metered:  46.8707294675
pc6:  1097HW  pc6_id Int64Index([6], dtype='int64')  pc6std:  6.97592141958  pc6_metered:  55.8277931836
pc6:  1097HZ  pc6_id Int64Index([7], dtype='int64')  pc6std:  12.6087673048  pc6_metered:  99.3890649374
pc6:  1097JE  pc6_id Int64Index([8], dtype='int64')  pc6std:  7.56191258563  pc6_metered:  62.1893887863
pc6:  1097JG  pc6_id Int64Index([9], dtype='int64')  pc6

pc6:  1097NS  pc6_id Int64Index([21], dtype='int64')  pc6std:  12.5003823662  pc6_metered:  96.4471561451
pc6:  1097PE  pc6_id Int64Index([22], dtype='int64')  pc6std:  10.1766451745  pc6_metered:  74.2173612219
pc6:  1097PK  pc6_id Int64Index([23], dtype='int64')  pc6std:  19.5375306781  pc6_metered:  143.363477402
pc6:  1097PL  pc6_id Int64Index([24], dtype='int64')  pc6std:  9.06117905507  pc6_metered:  74.6197613175
pc6:  1097PN  pc6_id Int64Index([25], dtype='int64')  pc6std:  11.3606041596  pc6_metered:  85.5230307636
pc6:  1097PT  pc6_id Int64Index([26], dtype='int64')  pc6std:  9.18307864602  pc6_metered:  69.8001497928
pc6:  1097PV  pc6_id Int64Index([27], dtype='int64')  pc6std:  9.58252866237  pc6_metered:  73.7369681585
pc6:  1097PW  pc6_id Int64Index([28], dtype='int64')  pc6std:  12.1441141029  pc6_metered:  92.7710494899
pc6:  1097RA  pc6_id Int64Index([29], dtype='int64')  pc6std:  19.3606938317  pc6_metered:  151.272811215
pc6:  1097RG  pc6_id Int64Index([30], dtype='i

pc6:  1097SG  pc6_id Int64Index([44], dtype='int64')  pc6std:  26.3234336474  pc6_metered:  191.724736161
pc6:  1097SJ  pc6_id Int64Index([45], dtype='int64')  pc6std:  27.1687580019  pc6_metered:  229.629735725
pc6:  1097SK  pc6_id Int64Index([46], dtype='int64')  pc6std:  26.0166133738  pc6_metered:  198.335584994
pc6:  1097SL  pc6_id Int64Index([47], dtype='int64')  pc6std:  17.9735339488  pc6_metered:  138.861038704
pc6:  1094SH  pc6_id Int64Index([48], dtype='int64')  pc6std:  4.98707129357  pc6_metered:  37.2243577862
pc6:  1094NN  pc6_id Int64Index([49], dtype='int64')  pc6std:  6.74029740182  pc6_metered:  51.5595970869
pc6:  1094ME  pc6_id Int64Index([50], dtype='int64')  pc6std:  5.40921593661  pc6_metered:  38.1841981438
pc6:  1094SB  pc6_id Int64Index([51], dtype='int64')  pc6std:  9.79359231027  pc6_metered:  68.9207740741
pc6:  1094SR  pc6_id Int64Index([52], dtype='int64')  pc6std:  6.44627874835  pc6_metered:  47.1862084055
pc6:  1094SP  pc6_id Int64Index([53], dtype='i

pc6:  1097RP  pc6_id Int64Index([36], dtype='int64')  pc6std:  26.5424912237  pc6_metered:  254.040810705
pc6:  1097RR  pc6_id Int64Index([37], dtype='int64')  pc6std:  26.1454348653  pc6_metered:  195.605504697
pc6:  1097RS  pc6_id Int64Index([38], dtype='int64')  pc6std:  22.4444108576  pc6_metered:  168.47417409
pc6:  1097RT  pc6_id Int64Index([39], dtype='int64')  pc6std:  27.5561866259  pc6_metered:  189.991219659
pc6:  1097RW  pc6_id Int64Index([40], dtype='int64')  pc6std:  20.8257328653  pc6_metered:  144.054112854
pc6:  1097RZ  pc6_id Int64Index([41], dtype='int64')  pc6std:  20.5463389136  pc6_metered:  148.51884879
pc6:  1097SE  pc6_id Int64Index([42], dtype='int64')  pc6std:  25.1371066061  pc6_metered:  172.30338659
pc6:  1097SG  pc6_id Int64Index([43], dtype='int64')  pc6std:  26.3234336474  pc6_metered:  188.839849849
pc6:  1097SJ  pc6_id Int64Index([44], dtype='int64')  pc6std:  27.1687580019  pc6_metered:  186.875384117
pc6:  1097SK  pc6_id Int64Index([45], dtype='int6

pc6:  1097RJ  pc6_id Int64Index([30], dtype='int64')  pc6std:  14.0323260038  pc6_metered:  97.6935659552
pc6:  1097RK  pc6_id Int64Index([31], dtype='int64')  pc6std:  15.2310327107  pc6_metered:  94.9278289755
pc6:  1097RL  pc6_id Int64Index([32], dtype='int64')  pc6std:  20.0381301008  pc6_metered:  139.408097603
pc6:  1097RM  pc6_id Int64Index([33], dtype='int64')  pc6std:  25.3796942365  pc6_metered:  181.046116319
pc6:  1097RN  pc6_id Int64Index([34], dtype='int64')  pc6std:  18.8609075424  pc6_metered:  137.165456746
pc6:  1097RP  pc6_id Int64Index([35], dtype='int64')  pc6std:  26.5424912237  pc6_metered:  173.225811836
pc6:  1097RR  pc6_id Int64Index([36], dtype='int64')  pc6std:  26.1454348653  pc6_metered:  187.485573585
pc6:  1097RS  pc6_id Int64Index([37], dtype='int64')  pc6std:  22.4444108576  pc6_metered:  161.06083215
pc6:  1097RT  pc6_id Int64Index([38], dtype='int64')  pc6std:  27.5561866259  pc6_metered:  188.679930567
pc6:  1097RW  pc6_id Int64Index([39], dtype='in

pc6:  1097PN  pc6_id Int64Index([23], dtype='int64')  pc6std:  11.3606041596  pc6_metered:  77.3616509233
pc6:  1097PT  pc6_id Int64Index([24], dtype='int64')  pc6std:  9.18307864602  pc6_metered:  59.8761829175
pc6:  1097PV  pc6_id Int64Index([25], dtype='int64')  pc6std:  9.58252866237  pc6_metered:  62.7916369474
pc6:  1097PW  pc6_id Int64Index([26], dtype='int64')  pc6std:  12.1441141029  pc6_metered:  78.466024523
pc6:  1097RA  pc6_id Int64Index([27], dtype='int64')  pc6std:  19.3606938317  pc6_metered:  127.350125727
pc6:  1097RG  pc6_id Int64Index([28], dtype='int64')  pc6std:  27.5360030176  pc6_metered:  190.850376875
pc6:  1097RH  pc6_id Int64Index([29], dtype='int64')  pc6std:  11.8945175854  pc6_metered:  74.994121727
pc6:  1097RJ  pc6_id Int64Index([30], dtype='int64')  pc6std:  14.0323260038  pc6_metered:  93.6335736038
pc6:  1097RK  pc6_id Int64Index([31], dtype='int64')  pc6std:  15.2310327107  pc6_metered:  99.928648062
pc6:  1097RL  pc6_id Int64Index([32], dtype='int6

In [146]:
pc6TrainedCounts

{'1091KZ': ['2012', '2013', '2014', '2015'],
 '1091LC': ['2012', '2013', '2014', '2015'],
 '1091LE': ['2012', '2013', '2014', '2015'],
 '1091LG': ['2012', '2013', '2014', '2015'],
 '1091LH': ['2012', '2013', '2014', '2015'],
 '1091LN': ['2012', '2013', '2014', '2015'],
 '1091LP': ['2012', '2013', '2014'],
 '1091LR': ['2012', '2013', '2014', '2015'],
 '1091ME': ['2012', '2013', '2014', '2015'],
 '1091MG': ['2012', '2013', '2014'],
 '1091NB': ['2012', '2013', '2014', '2015'],
 '1091NH': ['2012', '2013', '2014', '2015'],
 '1091NJ': ['2012', '2013', '2014', '2015'],
 '1091NP': ['2012', '2013', '2014', '2015'],
 '1091NR': ['2012', '2013', '2014', '2015'],
 '1091NV': ['2012', '2013', '2014', '2015'],
 '1091PA': ['2012', '2013', '2014', '2015'],
 '1091PB': ['2012', '2013', '2014', '2015'],
 '1091PD': ['2012', '2013', '2014', '2015'],
 '1091PG': ['2012', '2013', '2014', '2015'],
 '1091PH': ['2012', '2013', '2014', '2015'],
 '1091PK': ['2012', '2013'],
 '1091PP': ['2012', '2013', '2014', '2015'

In [149]:
inputSets = inputCombination(p_Tmin, p_Ninf)
pickUpInputCombination(pc6_prob, pc6TrainedCounts, pc6_archetype, inputSets, conn)